# Introduction

Make sure to fill out all of the fields you can.  

*   If you want to save your work click File --> Save As and save your own copy of this notebook.
*   All of the necessary files for this demonstration can be downloaded [here](https://byu.box.com/s/le7kyjbf80sfyfijyv1k0ov3go6kjdfe).
*   Event Date and Additional Model Notes are optional.
*   Include the file extension where relevant (target_db_filename should have the extension .db, csv_name, rating_curve_filename, and containing_huc8_filename should have the extention .csv).
*  Downstream_reach_id and containing_huc_8 can be found from the linked colab notebook at this link ([NWM Data Finder](https://colab.research.google.com/drive/18CAyUd4ffUoNWDLvbaBVTw0XVSQrHNWI?usp=sharing)) or simply input in the right format if they are already known.
* Make sure to download any data you want to save from the Google Colab file explorer when you are done, otherwise it will get deleted when the runtime expires


# Adding Data to the Database

In [ ]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import sqlite3

This step requires you to upload a correctly formatted csv of your input data if you choose that option, otherwise a csv file will be generated to save your input data.
*  If you choose the Existing Database option under DatabaseType you can upload an existing database with the same format as this database that you wish to add data to.

In [ ]:
# @title General Input Data
# Input to Populate fim_source Table
# @markdown Change the input_type parameter to CSV File if you want to populate the fields from a csv file.
input_type = 'CSV File' # @param ["CSV File", "Field Input"]
DatabaseType = 'New Database' # @param ["New Database", "Existing Database"]
target_db_filename = 'MississippiFIMDatabase.db' # @param {type:"string"}
# @markdown Ignore this field if your input_type is not CSV File.
csv_name = 'FIM_input_data.csv' # @param {type:"string"}
# @markdown Fill out the following fields if you chose the Field Input Option:
FIMSourceName = 'MSTallahatchieRiver_HAND' # @param {type:"string"}
CoordinateReference = 'EPSG:4326' # @param {type:"string"}
EntityName = 'Aquaveo LLC' # @param {type:"string"}
EntityContactEmail = 'azundel@aquaveo.com' # @param {type:"string"}
VersionNumber = '1.0' # @param {type:"string"}
YearCreated = 2021 # @param {type:"integer"}
EventDate = "" # @param {type:"date"}
AdditionalModelNotes = 'This model was created by the FHWA.' # @param {type:"string"}
# Input to Populate model_type Table
Software = 'SRH-2D' # @param ["HEC RAS-1D", "HEC RAS-2D", "HEC RAS-1D/2D Combo", "SRH-2D", "FIER", "AutoRoute", "HAND", "TRITON", "Satellite Observations", "Surveyed Flood Extents", "Other"]

blue = '\033[94m'
italics = '\033[3m'
end = '\033[0m'

if input_type == 'CSV File':
  if not os.path.isfile(csv_name):
    raise ValueError('CSV input file not found. Make sure to upload your csv file to the files tab or choose the Field Input option for input_type')
  input_data_df = pd.read_csv(csv_name)
else:
  input_data = {
  "FIMSourceName": FIMSourceName,
  "CoordinateReference": CoordinateReference,
  "EntityName": EntityName,
  "EntityContactEmail": EntityContactEmail,
  "VersionNumber": VersionNumber,
  "YearCreated": YearCreated,
  "EventDate": EventDate,
  "AdditionalModelNotes": AdditionalModelNotes,
  "Software": Software
  }
  input_data_df = pd.DataFrame(input_data, index = [0])
  input_data_df.to_csv('FIM_input_data.csv', index=False)
  print('Inputs saved as '+ blue + italics + 'FIM_input_data.csv'+ end +'.')

database_name = target_db_filename


def check_database_exists(filename):
  if os.path.isfile(filename) and DatabaseType != 'Existing Database':
    raise ValueError('Database file already exists, please change DatabaseType to Existing Database')
  if not os.path.isfile(filename) and DatabaseType != 'New Database':
    raise ValueError('No Database file found, please upload a database file and change the database name to match the file name or select the New Database option for DatabaseType')

check_database_exists(target_db_filename)

# Create database structure
conn = sqlite3.connect(database_name)
dbml_script = """
Table fim_source {
  FIMSourceID integer [pk, increment]
  FIMSourceName text
  CoordinateReference text
  PrimaryRatingCurveID integer
  ResponsibleEntityID integer
  ModelTypeID integer
  VersionNumber tinytext
  YearCreated smallint
  EventDate datetime
  AdditionalModelNotes text
}

Table responsible_entities {
  ResponsibleEntityID integer [pk, increment]
  EntityName text
  EntityContactEmail text
}

Table feature_ids_fim_source {
  FeatureID integer
  RatingCurveID integer
  FimSourceID integer
  ReferenceNetwork text
}

Table model_type {
  ModelTypeID smallint [pk, increment]
  Software text
}

Table flows {
  RatingCurveID integer
  Flow float(2)
  Depth float(2)
  ReturnPeriod smallint
  FloodExtentVectorID integer
  DepthRasterID integer
  WSERasterID integer
}


Table files {
  FileID integer [pk, increment]
  FileName text
}


Table rating_curves {
  RatingCurveID integer [pk, increment]
  RatingCurve text
  FlowUnit tinytext
  DepthUnit tinytext
}

Table fim_source_huc8 {
  FIMSourceID integer
  HUCNumber integer
}
"""

def dbml_to_sqlite(dbml_script, database_name):
    table_statements = [stmt.strip() for stmt in dbml_script.split('Table') if stmt.strip()]
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    for table_statement in table_statements:
        table_name, columns = table_statement.split('{')[0].strip(), table_statement.split('{')[1].split('}')[0].strip()
        columns = ", ".join([col.strip() for col in columns.split('\n') if col.strip()])
        cursor.execute(f"CREATE TABLE {table_name} ({columns})")

    conn.commit()
    conn.close()

    print('New database ' + blue + italics + f'{database_name}' + end + ' successfully created.')

if DatabaseType == 'New Database':
  dbml_to_sqlite(dbml_script, database_name)
elif DatabaseType == 'Existing Database':
  print(f"Database Name: '{target_db_filename}")

def check_for_duplicate_fim_sources(fim_sources):
  fim_sources = input_data_df['FIMSourceName'].to_list()
  duplicates = []
  seen = set()
  existing_fim_sources = set(pd.read_sql_query("SELECT FIMSourceName FROM fim_source", conn)["FIMSourceName"].tolist())
  for fim_source in fim_sources:
    if fim_source in seen or fim_source in existing_fim_sources:
      duplicates.append(fim_source)
    else:
      seen.add(fim_source)
  return duplicates

duplicates = check_for_duplicate_fim_sources(input_data_df)
if duplicates:
  raise ValueError(f'Duplicate FIM Source Names found: {", ".join(duplicates)}')

# Code to add data to the database
def insert_data_into_table(conn, table_name, data_frame):
    data_frame.to_sql(table_name, conn, if_exists='append', index=False)

# Populate model_type table if necessary
model_type = {
    'ModelTypeID': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Software': ["HEC RAS-1D", "HEC RAS-2D", "HEC RAS-1D/2D Combo", "SRH-2D", "FIER", "AutoRoute", "HAND", "TRITON", "Satellite Observations", "Surveyed Flood Extents", "Other"]
}
model_type_number = pd.read_sql_query("""SELECT COUNT(*) FROM model_type""",conn)
if model_type_number.iloc[0,0] == 0:
  model_type_df = pd.DataFrame(model_type)
  insert_data_into_table(conn, "model_type", model_type_df)

# Add data to fim_source_table
num_fim_sources_input = len(input_data_df)
fim_source_numbers = list(range(0, num_fim_sources_input))
start_fim_source_number = pd.read_sql_query("""SELECT COUNT(*) FROM fim_source""",conn)
start_fim_source_number = start_fim_source_number.iloc[0,0]
fim_source_numbers = [num + start_fim_source_number for num in fim_source_numbers]
print(f"New FIM Source IDs assigned to Input FIM Sources: {fim_source_numbers}")
# Get ResponsibleEntityID
starting_entity_number = pd.read_sql_query("""SELECT COUNT(*) FROM responsible_entities""", conn).iloc[0,0]
for entity_name in input_data_df['EntityName'].unique():
  # Check each unique entity name to see if it is in the database, and if it isn't then add it
  responsible_entity_check_existing = pd.read_sql_query(
      f"SELECT ResponsibleEntityID FROM responsible_entities WHERE EntityName LIKE '{entity_name}'", conn)
  if responsible_entity_check_existing.empty:
    responsible_entity_num = starting_entity_number
    input_data_df.loc[input_data_df['EntityName'] == entity_name, 'ResponsibleEntityID'] = responsible_entity_num
    starting_entity_number += 1
  else:
    responsible_entity_num = responsible_entity_check_existing.iloc[0,0]
    input_data_df.loc[input_data_df['EntityName'] == entity_name, 'ResponsibleEntityID'] = responsible_entity_num
# Get ModelTypeID
model_type_data = input_data_df['Software']
for Software in input_data_df['Software'].unique():
  model_type_check_existing = pd.read_sql_query(f"SELECT ModelTypeID FROM model_type WHERE Software LIKE '{Software}'",conn)
  model_type_num = model_type_check_existing.iloc[0,0]
  input_data_df.loc[input_data_df['Software'] == Software, 'ModelTypeID'] = model_type_num
input_data_df = input_data_df.astype({'ModelTypeID': int,'ResponsibleEntityID': int})
input_data_df.loc[:, 'FIMSourceID'] = fim_source_numbers
formats = ['%m/%d/%Y', '%m/%d/%y']
for fmt in formats:
  try:
    input_data_df['EventDate'] = pd.to_datetime(input_data_df['EventDate'], format=fmt)
    break
  except ValueError:
    pass
fim_source_df = input_data_df[["FIMSourceID","FIMSourceName","CoordinateReference","ResponsibleEntityID","ModelTypeID","VersionNumber","YearCreated","EventDate","AdditionalModelNotes"]]
insert_data_into_table(conn, "fim_source", fim_source_df)

# Add data to responsible_entities table
responsible_entities_df = input_data_df[['ResponsibleEntityID','EntityName', 'EntityContactEmail']]
unique_entity_ids = responsible_entities_df.groupby('ResponsibleEntityID').head(1)
existing_ids_df = pd.read_sql_query("SELECT ResponsibleEntityID FROM responsible_entities", conn)
existing_ids = set(existing_ids_df['ResponsibleEntityID'].tolist())
data_to_insert = unique_entity_ids[~unique_entity_ids['ResponsibleEntityID'].isin(existing_ids)]
insert_data_into_table(conn, "responsible_entities", data_to_insert)

if num_fim_sources_input == 1:
  print(f'{num_fim_sources_input} FIM Source successfully added to the database')
else:
  print(f'{num_fim_sources_input} FIM Sources successfully added to the database')

New database MississippiFIMDatabase.db successfully created.
New FIM Source IDs assigned to Input FIM Sources: [0, 1]
2 FIM Sources successfully added to the database


In [ ]:
# @title Run this cell to view the fim_source table
pd.read_sql_query("""SELECT * from fim_source""",conn)

,FIMSourceID,FIMSourceName,CoordinateReference,PrimaryRatingCurveID,ResponsibleEntityID,ModelTypeID,VersionNumber,YearCreated,EventDate,AdditionalModelNotes
0,0,MSTallahatchieRiver_SRH2D,EPSG:4269,None,0,3,1,2021,None,This FIM source was created by the FHWA.
1,1,MSTallahatchieRiver_HAND,EPSG:4326,None,1,6,4,2023,None,This FIM source was run in the BYU hydroinform...


In [ ]:
# @title Add HUC8 Data
# @markdown You can get the HUC8 data from the colab notebook linked in the introduction if you don't already have it.
# @markdown Follow the template format for the containing huc8 csv file to ensure correct upload
containing_huc8_filename = 'containing_huc8s.csv' #@param {type: "string"}
target_FimSourceID = 1 # @param {type:"integer"}
# Add data to the fim_source_huc8 table
model_huc8_list = pd.read_csv(containing_huc8_filename)['HUC8'].to_list()
fim_source_huc8_dict = {'FIMSourceID': [], 'HUCNumber': []}
for huc8 in model_huc8_list:
  fim_source_huc8_dict['FIMSourceID'].append(target_FimSourceID)
  fim_source_huc8_dict['HUCNumber'].append(huc8)
fim_source_huc8_df = pd.DataFrame(fim_source_huc8_dict)
existing_huc8_data = pd.read_sql_query("SELECT HUCNumber, FIMSourceID FROM fim_source_huc8", conn)
merged_df = fim_source_huc8_df.merge(existing_huc8_data, how='inner', on=['HUCNumber', 'FIMSourceID'])
blue = '\033[94m'
italics = '\033[3m'
if not merged_df.empty:
  fim_source_huc8_df = fim_source_huc8_df[~fim_source_huc8_df['HUCNumber'].isin(merged_df['HUCNumber'])]
  if not fim_source_huc8_df.empty:
    insert_data_into_table(conn, "fim_source_huc8", fim_source_huc8_df)
    print('HUC8 data successfully added to FIM Source with ' + blue + italics + f'FIMSourceID {target_FimSourceID}')
  if fim_source_huc8_df.empty:
    print('HUC8 number is already associated with FIM Source')
else:
  insert_data_into_table(conn, "fim_source_huc8", fim_source_huc8_df)
  print('HUC8 data successfully added to FIM Source with ' + blue + italics + f'FIMSourceID {target_FimSourceID}')

HUC8 data successfully added to FIM Source with FIMSourceID 1


In [ ]:
# @title Run this cell to view the fim_source_huc8 table
pd.read_sql_query("""SELECT * from fim_source_huc8 ORDER BY FIMSourceID""",conn)

,FIMSourceID,HUCNumber
0,0,8030201
1,1,8030201


# Rating Curve Data Tools

This step requires you to upload a rating curve file and a feature_ids file, both of which should be in csv format.

In [ ]:
# @title Add Rating Curve Data
# @markdown You can get a feature_id file from the colab notebook linked at the top of this page if you don't already have it.
target_db_filename = 'MississippiFIMDatabase.db' # @param {type:"string"}
target_FimSourceID = 1 # @param {type:"integer"}
feature_id_filename = 'feature_ids (HAND).csv' # @param {type:"string"}
# @markdown Follow the template format for the rating curve file to ensure correct upload.
# @markdown Choose "Yes" on the primary rating curve option if this rating curve is for the furthest downstream reach_id of your model.
primary_rating_curve = 'Yes' # @param ["Yes", "No"]
# @markdown Choose yes on seperate_files if your rating curve is associated with files not already referenced in the database.
# @markdown Choose no if your rating curve references files already connected to another rating curve, then type in the RatingCurveID for that rating curve as source_RatingCurveID
seperate_files = 'Yes' # @param ["Yes", "No"]
source_RatingCurveID = 0 # @param {type:"integer"}
rating_curve_filename = 'MSTallahatchieRiver_HANDFlows.csv' # @param {type:"string"}
FlowUnit = 'cfs' # @param ["cfs", "cms"]
DepthUnit = 'ft' # @param ["ft", "m"]
NWMReferenceNetworkVersion = '3.0' # @param ["1.2", "2.0","2.1","3.0"]
conn = sqlite3.connect(target_db_filename)
cur = conn.cursor()
# Attempt to read in files early to prevent partial additions to the database
flows_df = pd.read_csv(rating_curve_filename)
feature_ids = pd.read_csv(feature_id_filename)['feature_id'].to_list()

def insert_data_into_table(conn, table_name, data_frame):
    data_frame.to_sql(table_name, conn, if_exists='append', index=False)

# Get RatingCurveID
rating_curve_number = pd.read_sql_query("""SELECT COUNT(*) FROM rating_curves""",conn).iloc[0,0]
# Add RatingCurveID to the fim_source table
if primary_rating_curve == 'Yes':
  query = f"UPDATE fim_source SET PrimaryRatingCurveID = {rating_curve_number} WHERE FimSourceID = '{target_FimSourceID}'"
  cur.execute(query)
# Add data to rating_curves table
rating_curves_df = pd.DataFrame([{'RatingCurveID':rating_curve_number,"RatingCurve":rating_curve_filename,'FlowUnit':FlowUnit,'DepthUnit':DepthUnit}])
insert_data_into_table(conn, "rating_curves", rating_curves_df)

# Add data to the flows table
if seperate_files == 'Yes': # This is if you want to make new files for each new file
  flows_df['RatingCurveID'] = rating_curve_number
  vector_file_id = pd.read_sql_query("""SELECT COUNT(*) FROM files""",conn).iloc[0,0]
  for idx, row in flows_df.iterrows():
    flows_df.at[idx, 'FloodExtentVectorID'] = vector_file_id
    vector_file_id += 1
    if not pd.isna(row['DepthRaster']):
      depth_file_id = vector_file_id
      flows_df.at[idx, 'DepthRasterID'] = depth_file_id
      vector_file_id += 1
    if not pd.isna(row['WSERaster']):
      if not pd.isna(row['DepthRaster']):
        wse_file_id = depth_file_id + 1
      else:
        wse_file_id = vector_file_id
      flows_df.at[idx, 'WSERasterID'] = wse_file_id
      vector_file_id = wse_file_id + 1

  for col in ['DepthRasterID', 'WSERasterID']:
    if col not in flows_df.columns:
      flows_df[col] = np.NaN
  flows_data_df = flows_df

  if 'DepthRasterID' not in flows_df.columns:
    if 'WSERasterID' not in flows_df.columns:
      flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID']]
    else:
      flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID','WSERasterID']]
  elif 'WSERasterID' not in flows_df.columns:
    if 'DepthRasterID' in flows_df.columns:
      flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID','DepthRasterID']]
  else:
    flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID','DepthRasterID','WSERasterID']]
    flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID','DepthRasterID','WSERasterID']]
  insert_data_into_table(conn, "flows", flows_df)
  # Add data to the files table
  files_dict = {'FileID': [], 'FileName': []}
  for idx, row in flows_data_df.iterrows():
    vector_extent = row['VectorExtent']
    depth_raster = row['DepthRaster']
    wse_raster = row['WSERaster']
    vector_id = row['FloodExtentVectorID']
    depth_id = row['DepthRasterID']
    wse_id = row['WSERasterID']
    files_dict['FileID'].append(vector_id)
    files_dict['FileName'].append(vector_extent)
    if pd.notna(depth_id):
      files_dict['FileID'].append(depth_id)
      files_dict['FileName'].append(depth_raster)
    if pd.notna(wse_id):
      files_dict['FileID'].append(wse_id)
      files_dict['FileName'].append(wse_raster)
  files_df = pd.DataFrame(files_dict)
  insert_data_into_table(conn, "files", files_df)
else: #This is if you want to associate a rating curve with existing files
  old_curve = pd.read_sql_query(f'SELECT * from flows WHERE RatingCurveID = {source_RatingCurveID}',conn)
  new_curve = pd.read_csv(rating_curve_filename)
  new_flow_count = len(rating_curve_filename)
  new_curve['FloodExtentVectorID'] = old_curve['FloodExtentVectorID']
  new_curve['DepthRasterID'] = old_curve['DepthRasterID']
  new_curve['WSERasterID'] = old_curve['WSERasterID']
  new_curve = new_curve[['Flow','Depth','ReturnPeriod','FloodExtentVectorID','DepthRasterID','WSERasterID']]
  insert_data_into_table(conn, "flows", new_curve)

# Add data to the feature_ids_fim_source table
feature_ids_dict = {'FeatureID': [], 'RatingCurveID': [], 'FIMSourceID':[], 'ReferenceNetwork':[]}
for feature_id in feature_ids:
  feature_ids_dict['FeatureID'].append(feature_id)
  feature_ids_dict['RatingCurveID'].append(rating_curve_number)
  feature_ids_dict['FIMSourceID'].append(target_FimSourceID)
  feature_ids_dict['ReferenceNetwork'].append(NWMReferenceNetworkVersion)
feature_ids_fim_source_df = pd.DataFrame(feature_ids_dict)
insert_data_into_table(conn, "feature_ids_fim_source", feature_ids_fim_source_df)

blue = '\033[94m'
italics = '\033[3m'
end = '\033[0m'
print('Rating Curve ' + blue + italics + f'{rating_curve_filename}' + end + f' successfully added to FIM Source with FIMSourceID {target_FimSourceID}')

Rating Curve MSTallahatchieRiver_HANDFlows.csv successfully added to FIM Source with FIMSourceID 1


In [ ]:
# @title Run this cell to see the rating_curves table
pd.read_sql_query("""SELECT * from rating_curves ORDER BY RatingCurveID""",conn)

,RatingCurveID,RatingCurve,FlowUnit,DepthUnit
0,0,MSTallahatchieRiver_SRH2DFlows.csv,cfs,ft
1,1,MSTallahatchieRiver_HANDFlows.csv,cfs,ft


In [ ]:
# @title Replace Rating Curve Data
target_db_filename = 'Test.db' # @param {type:"string"}
target_FimSourceID = 0 # @param {type:"integer"}
target_RatingCurveID = 0 # @param {type:"integer"}
# @markdown You can get downstream_feature_id from the colab notebook linked at the top of this page if you don't already have it.
# @markdown The feature_id file containing all of the feature ids the model is related to can be obtained in the same place.
downstream_feature_id = 15290462 # @param {type:"integer"}
feature_id_filename = 'feature_ids.csv' # @param {type:"string"}
# @markdown Follow the template format for the rating curve file to ensure correct upload.
# @markdown Choose "Yes" on the primary rating curve option if this rating curve is for the furthest downstream feature_id of your model.
primary_rating_curve = 'Yes' # @param ["Yes", "No"]
rating_curve_filename = 'MSTallahatchieRiver_SRH2DFlows2.csv' # @param {type:"string"}
FlowUnit = 'cfs' # @param ["cfs", "cms"]
DepthUnit = 'ft' # @param ["ft", "m"]

# Attempt to read in files early to prevent partial additions to database
flows_df = pd.read_csv(rating_curve_filename)
feature_ids = pd.read_csv(feature_id_filename)['feature_id'].to_list()

conn = sqlite3.connect(target_db_filename)
cur = conn.cursor()

def insert_data_into_table(conn, table_name, data_frame):
    data_frame.to_sql(table_name, conn, if_exists='append', index=False)
def insert_data_into_table_replace(conn, table_name, data_frame):
    data_frame.to_sql(table_name, conn, if_exists='replace', index=False)

delete_query = "DELETE FROM rating_curves WHERE RatingCurveID = ?"
cur.execute(delete_query, (target_RatingCurveID,))

# Find each FileID in the files table that is associated with the a flow associated with Rating CurveID and delete it
delete_query1 = """ DELETE FROM files WHERE FileID IN (SELECT FloodExtentVectorID  FROM flows  WHERE RatingCurveID = ?) """
delete_query2 = """ DELETE FROM files WHERE FileID IN (SELECT DepthRasterID  FROM flows  WHERE RatingCurveID = ?) """
delete_query3 = """ DELETE FROM files WHERE FileID IN (SELECT WSERasterID  FROM flows  WHERE RatingCurveID = ?) """
cur.execute(delete_query1, (target_RatingCurveID,))
cur.execute(delete_query2, (target_RatingCurveID,))
cur.execute(delete_query3, (target_RatingCurveID,))

delete_query = "DELETE FROM flows WHERE RatingCurveID = ?"
cur.execute(delete_query, (target_RatingCurveID,))

# Set RatingCurveID
rating_curve_number = target_RatingCurveID
# Add RatingCurveID to the fim_source table
if primary_rating_curve == 'Yes':
  query = f"UPDATE fim_source SET PrimaryRatingCurveID = {rating_curve_number} WHERE FimSourceID = '{target_FimSourceID}'"
  cur.execute(query)
# Add data to rating_curves table
rating_curves_df = pd.DataFrame([{'RatingCurveID':rating_curve_number,"RatingCurve":rating_curve_filename,'FlowUnit':FlowUnit,'DepthUnit':DepthUnit}])
insert_data_into_table(conn, "rating_curves", rating_curves_df)

# Add data to the flows table
flows_df['RatingCurveID'] = rating_curve_number
new_flow_count = len(flows_df)
# Query the files tale and see how many there are in the database already
query = "SELECT COUNT(*) FROM files"
cur.execute(query)
existing_files_count = cur.fetchone()[0]

# Find each FileID already existing in the files table
query1 = """SELECT FloodExtentVectorID FROM flows WHERE RatingCurveID != ?"""
query2 = """SELECT DepthRasterID FROM flows WHERE RatingCurveID != ?"""
query3 = """SELECT WSERasterID FROM flows WHERE RatingCurveID != ?"""
flood_extent_vector_data = cur.execute(query1, (target_RatingCurveID,)).fetchall()
flood_extent_vector_ids = [element[0] for element in flood_extent_vector_data]
depth_raster_data = cur.execute(query2, (target_RatingCurveID,)).fetchall()
depth_raster_ids = [element[0] for element in depth_raster_data]
wse_raster_data = cur.execute(query3, (target_RatingCurveID,)).fetchall()
wse_raster_ids = [element[0] for element in wse_raster_data]
file_ids = flood_extent_vector_ids + depth_raster_ids + wse_raster_ids
file_ids = sorted(file_ids, key=lambda x: int(x))
# Determine how many new ids to add
flood_extent_vector_count = flows_df['VectorExtent'].count()
depth_raster_count = flows_df['DepthRaster'].count()
wse_raster_count = flows_df['WSERaster'].count()
new_ids_count = flood_extent_vector_count + depth_raster_count + wse_raster_count
if existing_files_count == 0:
  new_file_ids = [i for i in range(new_ids_count)]
else:
  new_file_ids = [i for i in range(len(file_ids)+new_ids_count)]
new_file_ids = [i for i in new_file_ids if i not in file_ids]
new_file_ids = sorted(new_file_ids, key=lambda x: int(x))
file_id_counter = 0
for idx, row in flows_df.iterrows():
  flows_df.at[idx, 'FloodExtentVectorID'] = new_file_ids[file_id_counter]
  file_id_counter += 1
  if not pd.isna(flows_df.at[idx, 'DepthRaster']):
    flows_df.at[idx, 'DepthRasterID'] = new_file_ids[file_id_counter]
    file_id_counter += 1
  if not pd.isna(flows_df.at[idx, 'WSERaster']) and not pd.isna(flows_df.at[idx, 'DepthRaster']):
    flows_df.at[idx, 'WSERasterID'] = new_file_ids[file_id_counter]
    file_id_counter += 1
  elif not pd.isna(flows_df.at[idx, 'WSERaster']):
    flows_df.at[idx, 'WSERasterID'] = new_file_ids[file_id_counter]
    file_id_counter += 1
flows_df = flows_df.astype({'FloodExtentVectorID': int,'DepthRasterID': int,'WSERasterID': int,})
flows_data_df = flows_df
flows_df = flows_df[['RatingCurveID','Flow','Depth','ReturnPeriod','FloodExtentVectorID','DepthRasterID','WSERasterID']]
insert_data_into_table(conn, "flows", flows_df)

# Add data to the files table
files_dict = {'FileID': [], 'FileName': []}
for idx, row in flows_data_df.iterrows():
  vector_extent = row['VectorExtent']
  depth_raster = row['DepthRaster']
  wse_raster = row['WSERaster']
  vector_id = row['FloodExtentVectorID']
  depth_id = row['DepthRasterID']
  wse_id = row['WSERasterID']
  files_dict['FileID'].append(vector_id)
  files_dict['FileName'].append(vector_extent)
  if pd.notna(depth_id):
    files_dict['FileID'].append(depth_id)
    files_dict['FileName'].append(depth_raster)
  if pd.notna(wse_id):
    files_dict['FileID'].append(wse_id)
    files_dict['FileName'].append(wse_raster)
files_df = pd.DataFrame(files_dict)
insert_data_into_table(conn, "files", files_df)

# Add data to the feature_ids_fim_source table
query5 = """DELETE FROM feature_ids_fim_source WHERE RatingCurveID = ?"""
cur.execute(query5,(rating_curve_number,))
feature_ids_dict = {'FeatureID': [], 'RatingCurveID': [], 'FIMSourceID':[], 'ReferenceNetwork':[]}
for feature_id in feature_ids:
  feature_ids_dict['FeatureID'].append(feature_id)
  feature_ids_dict['RatingCurveID'].append(rating_curve_number)
  feature_ids_dict['FIMSourceID'].append(target_FimSourceID)
  feature_ids_dict['ReferenceNetwork'].append(NWMReferenceNetworkVersion)
feature_ids_fim_source_df = pd.DataFrame(feature_ids_dict)
insert_data_into_table(conn, "feature_ids_fim_source", feature_ids_fim_source_df)

blue = '\033[94m'
italics = '\033[3m'
end = '\033[0m'
print('Rating Curve ' + blue + italics + f'{rating_curve_filename}' + end + f' successfully added to FIM Source with FIMSourceID {target_FimSourceID}, replacing RatingCurve with ' + blue + italics + f'RatingCurveID {target_RatingCurveID}')

Rating Curve MSTallahatchieRiver_SRH2DFlows2.csv successfully added to FIM Source with FIMSourceID 0, replacing RatingCurve with RatingCurveID 0


In [ ]:
# @title Delete Rating Curve Data
target_RatingCurveID = 1 # @param {type:"integer"}
check = pd.read_sql_query(f"SELECT COUNT(*) FROM rating_curves WHERE RatingCurveID = {target_RatingCurveID}",conn).iloc[0,0]
if check == 0:
  raise ValueError(f'Rating Curve with RatingCurveID {target_RatingCurveID} does not exist in the database')

delete_query = "DELETE FROM rating_curves WHERE RatingCurveID = ?"
cur.execute(delete_query, (target_RatingCurveID,))
# Find each file in the files table that is associated with the rating curve and delete it
delete_query1 = """ DELETE FROM files WHERE FileID IN (SELECT FloodExtentVectorID  FROM flows  WHERE RatingCurveID = ?) """
delete_query2 = """ DELETE FROM files WHERE FileID IN (SELECT DepthRasterID  FROM flows  WHERE RatingCurveID = ?) """
delete_query3 = """ DELETE FROM files WHERE FileID IN (SELECT WSERasterID  FROM flows  WHERE RatingCurveID = ?) """
cur.execute(delete_query1, (target_RatingCurveID,))
cur.execute(delete_query2, (target_RatingCurveID,))
cur.execute(delete_query3, (target_RatingCurveID,))
# Delete entries from reach_ids_fim_source related to the rating curve
query5 = """DELETE FROM feature_ids_fim_source WHERE RatingCurveID = ?"""
cur.execute(query5,(target_RatingCurveID,))
# Delete entries from flows related to the rating curve
delete_query = "DELETE FROM flows WHERE RatingCurveID = ?"
cur.execute(delete_query, (target_RatingCurveID,))
blue = '\033[94m'
italics = '\033[3m'
end = '\033[0m'
print('Rating Curve with ' + blue + italics + f'RatingCurveID {target_RatingCurveID}' + end + f' successfully deleted from the database')

Rating Curve with RatingCurveID 1 successfully deleted from the database


An easy way to view the entire database is to download it and then upload it to a sqlite viewer. To do this click on the Google Colab file explorer (file icon) on the left hand side of this webpage, click the three dots by the database file, and click download. Then go to the webpage https://sqliteviewer.app/ and upload your database file to view it.

# View Additional Database Tables

If you want to view the contents of individual tables you can use the cells below which contain SQL code to view each table not already shown in the cells above.

In [ ]:
pd.read_sql_query("""SELECT * from feature_ids_fim_source ORDER BY RatingCurveID""",conn)

,FeatureID,RatingCurveID,FimSourceID,ReferenceNetwork
0,15290452,0,0,3.0
1,15290450,0,0,3.0
2,15290454,0,0,3.0
3,15290448,0,0,3.0
4,15289244,0,0,3.0
...,...,...,...,...
15845,15290150,1,1,3.0
15846,15290150,1,1,3.0
15847,15290150,1,1,3.0
15848,15290150,1,1,3.0


In [ ]:
pd.read_sql_query("""SELECT * from files ORDER BY FileID""",conn)

,FileID,FileName
0,0,MSTallahatchieRiver_Q21930.shp
1,1,MSTallahatchieRiver_13.3_Q21930Depth.tif
2,2,MSTallahatchieRiver_13.3_Q21930WSE.tif
3,3,MSTallahatchieRiver_Q21340.shp
4,4,MSTallahatchieRiver_13.3_Q21340Depth.tif
5,5,MSTallahatchieRiver_13.3_Q21340WSE.tif
6,6,MSTallahatchieRiver_Q100.shp
7,7,MSTallahatchieRiver_13.3Q100Depth.tif
8,8,MSTallahatchieRiver_13.3Q100WSE.tif
9,9,MSTallahatchieRiver_Q18370.shp


In [ ]:
pd.read_sql_query("""SELECT * from flows ORDER BY RatingCurveID, Flow DESC""",conn)

,RatingCurveID,Flow,Depth,ReturnPeriod,FloodExtentVectorID,DepthRasterID,WSERasterID
0,0,21930.0,9.410,NaN,0,1.0,2.0
1,0,21340.0,9.309,NaN,3,4.0,5.0
2,0,19700.0,9.023,100.0,6,7.0,8.0
3,0,18370.0,8.784,NaN,9,10.0,11.0
4,0,17780.0,8.675,NaN,12,13.0,14.0
5,0,16160.0,8.370,50.0,15,16.0,17.0
6,0,14220.0,7.986,NaN,18,19.0,20.0
7,0,12840.0,7.699,25.0,21,22.0,23.0
8,0,10670.0,7.210,NaN,24,25.0,26.0
9,0,7800.0,6.013,10.0,27,28.0,29.0


In [ ]:
pd.read_sql_query("""SELECT * from responsible_entities ORDER BY ResponsibleEntityID""",conn)

,ResponsibleEntityID,EntityName,EntityContactEmail
0,0,Aquaveo LLC,azundel@aquaveo.com
1,1,Brigham Young University,samueljo@byu.edu


# Upload Database and Data to Hydroshare

Go to the [HydroShare website](https://www.hydroshare.org/home/) and create an account there which will allow you to upload your data.

In [ ]:
!pip install hsclient -q &> log.log
from google.colab import files
from hsclient import HydroShare
import json
import numpy as np
import os
import pandas as pd
import sqlite3

Run the code below to sign into HydroShare. Sign in using your HydroShare account credentials.

In [ ]:
hs = HydroShare()
hs.sign_in()

Username: sam.oldham
Password for sam.oldham: ··········


Fill in and run the cell below to create a csv list of the files you need to upload to HydroShare from a rating curve file.

* Make sure to upload your rating curve file to the file explorer.

* After running this cell make sure to upload all of your raster and vector files to the file explorer. Before uploading ensure that your shapefiles and rasters are in the correct projection (EPSG: 4326). This will allow them to be visualized through HydroShare.

* Make sure to upload all of the files associated with a shapefile, not just the one with the .shp extension.

In [ ]:
# read in the rating_curves.csv file, then make a list of all the values under the VectorExtent, DepthRaster, and WSERaster columns
rating_curve_csv = 'MSTallahatchieRiver_SRH2DFlows.csv' #@param {type:"string"}
db_file = 'FIMDatabase.db' #@param {type:"string"}
name_file = pd.read_csv(rating_curve_csv)
filenames = name_file['VectorExtent'].to_list()
filenames += name_file['DepthRaster'].to_list()
filenames += name_file['WSERaster'].to_list()
filenames = list(set(filenames))
filenames.append(db_file)
filenames.append(rating_curve_csv)
json_filename = os.path.splitext(db_file)[0]+'.json'
filenames.append(json_filename)
filenames.append("README.txt")
filenames = [x for x in filenames if str(x) != 'nan']
filenames_df = pd.DataFrame(filenames)
filenames_df.columns = ['filename']
filenames_df.to_csv('filenames.csv', index=False)
print("csv file 'filenames.csv' added to the Google Colab file explorer")

# Create a README.txt file
with open("README.txt", 'w') as f:
    f.write(f"""The database file is the {db_file} file, and it is also available as a JSON file as {json_filename}.

The rating curves are the CSV files in the resource, and the individual rasters and shapefiles are the extent, depth, and water surface elevation files.

The file naming system is set up so that it is FIMSourceID-RatingCurveID-type-flow. Type is either Extent (EXT), Depth (DEP), or Water Surface Elevation (WSE).""")

print("README.txt file created successfully and added to the Google Colab file explorer")

csv file 'filenames.csv' added to the Google Colab file explorer
README.txt file created successfully and added to the Google Colab file explorer


Set the value of the variables in the following fields to include the metadata you wish to include in a new HydroShare resource if you don't already have an existing resource.

In [ ]:
resource_subjects = ['FIM','Flood Inundation Mapping','Flow', 'Discharge', 'River Flow']
resource_description = """This is a compilation of test FIM maps made from a model we received from USACE (ERDC) of the North Platte River. Please don't use this data except for testing purposes."""

Fill out the fields below or ignore them according to the instructions, then run the cell.

In [ ]:
resource_type = 'New' #@param ['New','Existing']
# Use this field if you have an existing resource you want to add or update the database in, otherwise leave it blank
filenames = 'filenames.csv' #@param {type:"string"}
db_file = 'NebraskaFIMDatabase.db' #@param {type:"string"}
#@markdown Fill in the resource_id if you have an existing HydroShare resource. This id is the end of the resource URL when you view the resource on HydroShare.
resource_id = "d105e214ddd840b09040b6e51246f187" #@param {type:"string"}
#@markdown Fill in resource_name to create a new HydroShare resource if you don't already have one.
resource_name = 'BYU FIM Database Nebraska USACE Model' #@param {type:"string"}

filenames = pd.read_csv(filenames)
# Connect to database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

def process_dataframe(df):
  """
  Creates a dataframe containing the data necessary to rename the file.
  Args:
      dataframe (pd.DataFrame): A DataFrame which contains the names of all the files for upload to HydroShare.
  Returns:
      dataframe (pd.DataFrame): The DataFrame with FIMSourceID, RatingCurveID, and flow for the each filename.
  """
  # Create a dictionary to store FileIDs
  file_id_dict = {}
  # Efficiently query for FileIDs using placeholders
  unique_filenames = df['filename'].unique()
  query = f"SELECT FileName, FileId FROM files WHERE FileName IN ({','.join('?' * len(unique_filenames))})"
  cursor.execute(query, tuple(unique_filenames))
  # Store FileIDs in the dictionary
  for row in cursor.fetchall():
    file_id_dict[row[0]] = row[1]
  # Add 'FileID' column based on the dictionary
  df['FileID'] = df['filename'].map(file_id_dict)
  # Create a dictionary to store associated flows
  flow_data_dict = {}
  # Define flow queries
  flow_queries = [
      "SELECT * FROM flows WHERE FloodExtentVectorID = ?",
      "SELECT * FROM flows WHERE DepthRasterID = ?",
      "SELECT * FROM flows WHERE WSERasterID = ?"
  ]

  # Loop through each file and query for associated flows
  for file_id in df['FileID'].unique():
    flow_data = None
    for query in flow_queries:
      cursor.execute(query, (file_id,))
      result = cursor.fetchone()
      if result:
        flow_data = result
        break  # Stop iterating through flow queries if data is found
    flow_data_dict[file_id] = flow_data
  # Merge flow data into the dataframe
  if flow_data_dict:
    flow_df = pd.DataFrame.from_dict(flow_data_dict, orient='index', columns=['RatingCurveID','flow','depth','rp','FloodExtentVectorID','DepthRasterID','WSERasterID'])
    joined_df = df.merge(flow_df, how='left', left_on='FileID', right_index=True)
  else:
    joined_df = df.copy()

  rating_curves_exist = pd.read_sql_query("""SELECT RatingCurveID, RatingCurve from rating_curves ORDER BY RatingCurveID""",conn)
  joined_df = joined_df.merge(rating_curves_exist, how='left', on='RatingCurveID')
  feature_ids_fim_source_exist = pd.read_sql_query("""SELECT * from feature_ids_fim_source ORDER BY RatingCurveID""",conn)
  full_data = joined_df.merge(feature_ids_fim_source_exist, how='inner', on='RatingCurveID')
  unique_data = []
  # Loop through each group defined by RatingCurveID and FIMSourceID
  for group_name, group_data in full_data.groupby(['RatingCurveID']):
    # Select the first row
    first_instance = group_data.iloc[0]
    # Append the first instance to the unique_data list
    unique_data.append(first_instance.copy())

  # Convert the list of DataFrames to a single DataFrame
  full_data_unique = pd.DataFrame(pd.concat(unique_data))
  full_data_unique = full_data_unique.T
  fim_source_df = full_data_unique[['RatingCurveID','FimSourceID']]
  fim_source_df = fim_source_df.drop_duplicates()
  joined_df['FIMSourceID'] = joined_df.apply(lambda row: fim_source_df.loc[fim_source_df['RatingCurveID'] == row['RatingCurveID'], 'FimSourceID'].values[0] if row['RatingCurveID'] in fim_source_df['RatingCurveID'].values else None, axis=1)
  joined_df = joined_df.sort_values(by=['FileID'])

  return joined_df

def create_filename(row):
  """
  Creates a new filename based FIMSourceID, RatingCurveID, data type, and flow.
  Example: 1-1-EXT-1000 (FIMSourceID-RatingCurveID-DataType(floodextentvector)-Flow)
  Args:
      row (pd.Series): A row from the DataFrame.
  Returns:
      str: The constructed new filename.
  """
  fim_source_id = int(row['FIMSourceID']) if pd.notna(row['FIMSourceID']) else None
  file_id = int(row['FileID']) if pd.notna(row['FileID']) else None
  rating_curve_id = int(row['RatingCurveID']) if pd.notna(row['RatingCurveID']) else None
  flood_extent_vector_id = int(row['FloodExtentVectorID']) if pd.notna(row['FloodExtentVectorID']) else None
  depth_raster_id = int(row['DepthRasterID']) if pd.notna(row['DepthRasterID']) else None
  wse_raster_id = int(row['WSERasterID']) if pd.notna(row['WSERasterID']) else None
  flow = int(row['flow']) if pd.notna(row['flow']) else None
  filename = row['filename']
  # Check if FileID is null and if it is don't rename the file as it is not an output file or is not in the database
  if pd.isna(file_id):
    return row['filename']
  # Construct filename
  filename_parts = [str(fim_source_id)]
  filename_parts += [str(rating_curve_id)]
  if flood_extent_vector_id == file_id:
    filename_parts += ['EXT']
  elif depth_raster_id == file_id:
    filename_parts += ['DEP']
  elif wse_raster_id == file_id:
    filename_parts += ['WSE']
  filename_parts.append(str(flow))
  filename_full = '-'.join(filename_parts)
  last_period_index = filename.rfind('.')
  filename_full += filename[last_period_index:]

  return filename_full

processed_dataframe = process_dataframe(filenames)
processed_dataframe['new_filename'] = processed_dataframe.apply(create_filename, axis=1)
processed_dataframe = processed_dataframe[['filename','new_filename']]
shp_rows = processed_dataframe[processed_dataframe['new_filename'].str.endswith('.shp')]
# Function to create auxiliary file entry
def create_auxiliary_entry(row, extension):
    """
    Creates a Series with entries for 'filename' and 'new_filename' with modified extensions.
    Args:
        row: A pandas Series representing a row in the DataFrame.
        extension: String representing the new extension (e.g., 'dbf', 'prj', 'shx').
    Returns:
        A pandas Series with modified filenames and new filenames.
    """
    return pd.Series({
        'filename': row['filename'].replace('.shp', f'.{extension}'),
        'new_filename': row['new_filename'].replace('.shp', f'.{extension}')
    })
auxiliary_dfs = []
for extension in ['dbf', 'prj', 'shx']:
    auxiliary_df = shp_rows.apply(lambda row: create_auxiliary_entry(row, extension), axis=1)
    auxiliary_dfs.append(auxiliary_df)

processed_dataframe = pd.concat([processed_dataframe] + auxiliary_dfs, ignore_index=True)
processed_dataframe.to_csv('new_filenames.csv', index=False)
print("""New filenames saved to 'new_filenames.csv' in the Google Colab files explorer.
Make sure to download this file for future reference""")
# Check if the files exist in the Colab file explorer before attempting to rename
file_check = processed_dataframe.copy()
file_check = processed_dataframe[processed_dataframe['filename'] != f'{db_file[:-3]}.json']
bad_files = []
for each in file_check['filename']:
  if not os.path.exists(each):
    bad_files.append(each)
if len(bad_files) > 0:
  for each in bad_files:
    print(f'Error: {each} does not exist')
  raise ValueError('Some necessary files are not in the Colab file explorer')
# Use the new filenames to rename the files before export
for index, row in processed_dataframe.iterrows():
  old_name = row['filename']
  new_name = row['new_filename']
  #check if old file name exists in the file explorer, and if it does rename the file
  if os.path.exists(old_name):
    os.rename(old_name, new_name)
def rename_files_in_database(dataframe, database_path):
  """
  Renames files in an SQLite database based on a DataFrame.
  Args:
      dataframe (pd.DataFrame): The DataFrame containing filename and new_filename columns.
      database_path (str): The path to the SQLite database.
  """
  # Update query
  update_query = """UPDATE files SET filename = ? WHERE filename = ?"""
  # Loop through DataFrame and update database
  for index, row in dataframe.iterrows():
    original_filename = row['filename']
    new_filename = row['new_filename']
    cursor.execute(update_query, (new_filename, original_filename))

  # Commit changes and close connection
  conn.commit()
  conn.close()
  return
rename_files_in_database(processed_dataframe, db_file)

def sqlite_to_json(db_filename, json_filename):
  data = {}
  conn = sqlite3.connect(db_filename)
  conn.row_factory = sqlite3.Row  # Use row factory for dictionary-like access

  cursor = conn.cursor()
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
  tables = [row['name'] for row in cursor.fetchall()]

  for table in tables:
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    data[table] = [dict(row) for row in rows]

  with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)
  conn.close()

json_filename = db_file[:-3] + '.json'
sqlite_to_json(db_file, json_filename)
print(f"Converted SQLite database '{db_file}' to JSON file '{json_filename}'")
new_filenames = processed_dataframe['new_filename'].to_list()
if resource_type == 'New':
  new_resource = hs.create()
  resIdentifier = new_resource.resource_id
  print('Your new resource is available at: ' +  str(new_resource.metadata.url))
  print('resource_id = '+ str(resIdentifier))
  new_resource.metadata.title = resource_name
  new_resource.metadata.abstract = resource_description
  new_resource.metadata.subjects = resource_subjects
  new_resource.save()
  resource = new_resource
else:
  resource = hs.resource(resource_id)
print("""
The code will most likely throw an error when it is done uploading the files
if you have a lot of files. After the error appears you should check to make
sure that your files have all uploaded by viewing your resource on HydroShare at:
"""
 + str(resource.metadata.url))
resource.file_upload(*new_filenames)

ValueError: Shape of passed values is (1, 1), indices imply (1, 7)

After uploading your data to HydroShare you will need to change the sharing status to public in order for your files to be able to be visualized using the visualization application.